In [2]:
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


### Read and Merge Data

In [17]:
nom_partidos = pd.read_csv('./data/NomPartidos', encoding='iso-8859-1')
mesas_presidente = pd.read_csv('./data/MesasCandidaturaPresidente', encoding='iso-8859-1')
nom_ambitos = pd.read_csv('./data/NomAmbitos', encoding='iso-8859-1')

nom_partidos.replace({'á': 'a','é': 'e','í': 'i','ó': 'o','ú': 'u'}, inplace=True, regex=True)
nom_ambitos.replace({'á': 'a','é': 'e','í': 'i','ó': 'o','ú': 'u'}, inplace=True, regex=True)

m1 = pd.merge(nom_partidos, mesas_presidente, left_on='parCodigo', right_on='vot_parCodigo')
votos_presidente_complete = pd.merge(m1, nom_ambitos, left_on=['vot_proCodigoProvincia','vot_depCodigoDepartamento'], right_on=['depCodigoProvincia', 'depCodigoDepartamento'])
votos_presidente_complete = pd.DataFrame(votos_presidente_complete)

columns = ['vot_proCodigoProvincia','depNombre', 'vot_munCodigoMunicipio', 'vot_mesCodigoCircuito', 'vot_mesCodigoMesa','parSiglas', 'vot_parCodigo', 'votVotosPartido']
votos_presidente = votos_presidente_complete[columns]

### Busqueda de mesas con posibles errores.

In [48]:
votos_presidente.parSiglas.unique()

array([u'MOVIMIENTO AL SOCIALISMO ', u'MOVIMIENTO DE ACCION VECI',
       u'PARTIDO POPULAR          ', u'ALIANZA FRENTE PARA LA VI',
       u'ALIANZA PROGRESISTAS     ', u'ALIANZA COMPROMISO FEDERA',
       u'MST - NUEVA IZQUIERDA    ', u'ALIANZA CAMBIEMOS        ',
       u'ALIANZA FRENTE POPULAR   ', u'ALIANZA FRENTE DE IZQUIER',
       u'ALIANZA UNIDOS POR UNA NU'], dtype=object)

##### Cantidad de Mesas con 0 votos registrados

In [50]:
print "Cant Mesas donde el FPV saco 0 votos: ", sum(votos_presidente[votos_presidente.parSiglas == "ALIANZA FRENTE PARA LA VI"].votVotosPartido == 0)
print "Cant Mesas donde Cambiemos saco 0 votos: ", sum(votos_presidente[votos_presidente.parSiglas == "ALIANZA CAMBIEMOS        "].votVotosPartido == 0)
print "Cant Mesas donde UNA saco 0 votos: ", sum(votos_presidente[votos_presidente.parSiglas == "ALIANZA UNIDOS POR UNA NU"].votVotosPartido == 0)

Cant Mesas donde el FPV saco 0 votos:  3697
Cant Mesas donde Cambiemos saco 0 votos:  3750
Cant Mesas donde UNA saco 0 votos:  4106


##### Listado de mesas donde hay 0 votos para esos partidos

In [57]:
mesas_sin_votos_fpv = votos_presidente[(votos_presidente.parSiglas == "ALIANZA FRENTE PARA LA VI") & (votos_presidente.votVotosPartido == 0)]
mesas_sin_votos_cambiemos = votos_presidente[(votos_presidente.parSiglas == "ALIANZA CAMBIEMOS        ") & (votos_presidente.votVotosPartido == 0)]
mesas_sin_votos_una = votos_presidente[(votos_presidente.parSiglas == "ALIANZA UNIDOS POR UNA NU") & (votos_presidente.votVotosPartido == 0)]

##### Top N Mesas con mayor cantidad de votos

In [67]:
cantidad_mesas = 10
votos_presidente.loc[votos_presidente.votVotosPartido.order(ascending=False).head(cantidad_mesas).index]

,vot_proCodigoProvincia,depNombre,vot_munCodigoMunicipio,vot_mesCodigoCircuito,vot_mesCodigoMesa,parSiglas,vot_parCodigo,votVotosPartido
176302,2,General Madariaga,47,0340,18,ALIANZA PROGRESISTAS,132,805
231994,2,Matanza,61,0635E,2720,ALIANZA FRENTE PARA LA VI,131,793
394175,2,San Isidro,106,0892,398,ALIANZA CAMBIEMOS,135,757
188595,2,General Pueyrredon,50,0364,112,ALIANZA CAMBIEMOS,135,682
339617,2,Moron,80,0673,662,ALIANZA CAMBIEMOS,135,678
207739,2,General San Martin,52,0388,913,ALIANZA UNIDOS POR UNA NU,138,660
281272,2,La Plata,63,0496,391,ALIANZA UNIDOS POR UNA NU,138,657
91411,2,Almirante Brown,3,0021A,888,ALIANZA CAMBIEMOS,135,645
101578,2,Avellaneda,4,0030,184,ALIANZA CAMBIEMOS,135,644
704958,10,El Carmen,36,0031,821,ALIANZA CAMBIEMOS,135,386


In [71]:
votos_presidente.votVotosPartido.describe()

count    1059707.000000
mean          20.892436
std           36.606952
min            0.000000
25%            0.000000
50%            2.000000
75%           24.000000
max          805.000000
Name: votVotosPartido, dtype: float64